# Modelling the Word Game Wordle

The following notebook was created by Ken Pierce. The intention here give some practice in identifying and including pre-conditions, post-conditions, and data invariants using assertions in Python. These are fundamental concepts in creation of formal models, and the use of Python assertions allows us to explore them in a familiar setting.

## Wordle

Wordle is a word game created by Josh Wardle (https://twitter.com/powerlanguish); it is now owned and published by the New York Times: https://www.nytimes.com/games/wordle/index.html. The instructions for Wordle are as follows:

* Guess the WORDLE in 6 tries.
* Each guess must be a valid 5 letter word. 
* The color of the tiles will change to show how close your guess was to the word.
* A green letter shows it is in the correct position.
* A yellow letter shows it is in the word but in the wrong position.
* A grey letter shows that it is not in the word in any position.

## Building Models

There is no right or wrong way to construct a programme (or formal specification) from natural language specifications. We need to consider the purpose, which guides the abstraction decisions. The steps however can be broadly broken down as:

1. Analyse the functional behaviour from the requirements
2. Extract a list of possible data types (often from nouns) and functions (often from actions)
3. Create a dictionary by giving explanations to items in the list
4. Sketch out data types
5. Sketch out functions
6. Refine and add restrictions
7. Review and refine

The following will guide us through these steps to define a Wordle game.

## Functional Behaviour and Data Types

From the description above, we can identify the key elements of the instructions to understand the functionality and suggest data types:

* There’s a secret **wordle**
    - It has **five** letters (we can infer this though it is not explicit)
    - The player loses after **six** tries
* The player can make a **guess**
    - Guess must be **five** letters
    - Guess must be **valid** (= real word?)
* The game should output for each letter in a guess:
    - If the letter is in the **correct position**
    - If the letter in the **wrong position** (but in the word)
    - If the letter is **not in the word**

# The Programme

## Setup

Here, we simply include some imports for the later, as well as download a list of valid 5-letter words. Note, if the download  does not work, you can simply define a list of custom list of 5-letter strings, i.e. `WORDS = ["HELLO", ...]`.

In [114]:
from dataclasses import dataclass   # for defining dataclasses
from enum import Enum               # for defining enumerations
import typing                       # for type hinting
import random                       # for selecting a random answer
from typing import List             # for defining custom typed lists
from urllib.request import urlopen  # to load web data

# Uncomment commented lines if SSL error
# import ssl

url = 'https://raw.githubusercontent.com/tabatkins/wordle-list/main/words'
# context = ssl._create_unverified_context()
# WORDS = [word.rstrip().decode('UTF-8').upper() for word in urlopen(url, context=context).readlines()]
WORDS = [word.rstrip().decode('UTF-8').upper() for word in urlopen(url).readlines()]



## Data Types and Contants

Next, we can define some simple data types, and constants that we will need to use, based on the above analysis. Here we define:

* The length of a valid word.
* The maximum number of guesses.
* A `Word` type as an alias of `str`
* An enumeration of the three types of clue (i.e. colours)
* An enumeration for the game state, either in play, won, or lost.

In [115]:
# constants
WORD_LENGTH = 5 
MAX_GUESSES = 6

# define Word as an alias for string
Word = str

# type enumerating the three possible clue colours
Clue = Enum('Clue', ['GREEN', 'YELLOW', 'GREY'])

# type enumerating the state of the game
Gamestate =  Enum('Gamestate', ['WON', 'LOST', 'PLAYING'])

## Guess

We can now introduce a `Guess` type, which represents a guess after it is processed by the game (i.e. one line of the output). We use the `dataclass' decorator and properties to simplify definition. This includes the word that was guessed, and the clues.

1. There are two invariants to add to the `setter` functions; what are these? Complete the assertions and include an appropriate message for when they fail. These should restrict the values that `word` and `clues` can take. 
2. Create some test values for `Guess` to demonstrate the invariants, i.e.  `g1 = Guess("HELLO", [Clue.YELLOW, ...])`.

In [116]:
@dataclass
class Guess:
    """
    A class to represent a guess in Wordle, which is a Word 
    plus a clue for each letter.
    """
    word: Word
    clues: List[Clue]

    @property
    def word(self) -> Word:
        return self._word

    @word.setter
    def word(self, word: Word):
        # invariant
        assert len(word) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.word) <> {WORD_LENGTH}"  
        self._word = word

    @property
    def clues(self) -> List[Clue]:
        return self._clues

    @clues.setter
    def clues(self, clues: Clue):
        # invariant
        assert len(clues) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.clues) <> {WORD_LENGTH}"              
        self._clues = clues

    def __repr__(self):
        """
        Custom representation for pretty printing.
        """
        cluestr = [str(self.word[i]) + ": " + \
            self.clues[i].name for i in range(WORD_LENGTH)]
        return f"{self.word}: {cluestr}"       

# g1 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
#g2 = Guess("HELL", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
#g3 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN]) 

# Auxiliary Functions

There are multiple ways to specify the game. The suggestion here is to have a main class `Game` that includes the state and top-level functions including `make_guess`. To help define those, we define two auxiliary functions:

* `check_letter`: Compute clue given a character, its position, and word
* `check_guess`: Compute clues given a word and the answer

## check_letter 

This function computes the clue (colour) for a given letter, given a word and the index of the letter in the word. We will start with a naïve implementation:

* If it's in the right place, return green (i.e. `Clue.GREEN`)
* If it’s not there, return grey
* Otherwise, return yellow.

Complete the function in the following way:

3. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
4. Complete the implementation based on the sketch above.
5. Include some test cases to check your implementation, e.g. `print(check_letter("S", 0, "STOUT"))`.

In [117]:
def check_letter(letter: str, index: int, word: Word) -> Clue:
    """
    Given a letter and an index, computes the colour of the blue
    based on the word.
    """
    # pre-condition
    assert 0 <= index < WORD_LENGTH and \
           len(word) == WORD_LENGTH, "pre-check_letter failed."
           
    if word[index] == letter: return Clue.GREEN  
    elif letter not in word: return Clue.GREY
    else: return Clue.YELLOW

print(check_letter("S", 0, "STOUT"))
print(check_letter("S", 3, "STOUT"))
print(check_letter("Z", 0, "STOUT"))

Clue.GREEN
Clue.YELLOW
Clue.GREY


## Check Guess

Given a word and the correct answer, this function computes the list of clues. This can be achieved by declaring a local variable, and for each letter in the guess, use `check_letter`; we can use `enumerate(guess)` to get the index.

6. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
7. Complete the implementation based on the sketch above.
8. Include some test cases to check your implementation, e.g. `print(check_guess("STAND", "STOUT"))`.

In [118]:

def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess, compute the list of 
    clues corresponding to each letter.
    """
    # pre-condition
    assert len(word) == WORD_LENGTH and \
           len(guess) == WORD_LENGTH, "pre-check_guess failed"
           
    clues = []
    for i,letter in enumerate(guess):
        clue = check_letter(guess[i], i ,word)
        clues.append(clue)
    return clues

print(check_guess("STAND", "STOUT"))

[<Clue.GREEN: 1>, <Clue.GREEN: 1>, <Clue.GREY: 3>, <Clue.GREY: 3>, <Clue.YELLOW: 2>]


# Game State and Top-level Functionality

Here we define a `Game` class; the state is the current wordle (`answer`), the guesses made so far, and the state of the game (playing, won, or lost). Functionlity is included to print a message to the player about the game, and to rest the game when it is over.

## make_guess

The top-level function is `make_guesss` which takes a single word as a parameter. This function should use `check_guesss` to compute the guess and store it in `guesses`; it should then update the game state by checking if the game has been won (the word is exactly the same as the answer), or lost (the player reached the maximum number of guesses).

9. What pre-conditions should be included? These should restrict the parameters and the game state in which a guess is made (e.g. the player should not be able to guess if the game finished). Add these and an appropriate message.
10. Complete the implementation based on the sketch above, and play a game! You can alternate `game.make_guess(...)` and `game.print_state()` to play.

Note, can you spot the problem with the naïve implementation of `check_letter`?

In [119]:

class Game:
    answer: Word
    guesses: List[Guess]
    gstate: Gamestate

    def __init__(self, answer="STOUT"):
        """
        Constructor for game.
        """
        self.answer = answer
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def make_guess(self, word: Word):
        """
        Make a guess at the wordle.
        """
        # make guesses uppercase
        word = word.upper() 

        # pre-condition
        assert self.gstate == Gamestate.PLAYING and \
               len(self.guesses) < MAX_GUESSES and \
               word in WORDS, "pre-guess failed."
               
        self.guesses.append(Guess(word, check_guess(self.answer, word)))
        if word == self.answer: self.gstate = Gamestate.WON
        elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

    def print_state(self):
        """
        Prints a message to the user based on the current state of the game.
        """
        for guess in self.guesses: print(guess)
        if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
        elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
        else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

    def game_over(self) -> bool:
        """
        Yields true if the game is over (won or lost), false otherwise.
        """
        return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 

    def reset(self):
        """
        Reset the game by picking a new word, clearing the guess, and
        setting the state back to playing.
        """
        # pre-condition
        assert self.game_over(), "Cannot reset, game in play"
        self.answer = random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING
   
game = Game()
game.print_state()
game.make_guess("stand")
game.print_state()


Guess the wordle, you have 6 guesses remaining.
STAND: ['S: GREEN', 'T: GREEN', 'A: GREY', 'N: GREY', 'D: GREY']
Guess the wordle, you have 5 guesses remaining.


# CSC2034 Wordle Project - Mark Hudson
My changes to the Wordle game below.

## Task 2 - Hint Function

In [120]:
# type string for hint
Hint = str

def hint(word: Word, guesses: List[Guess]) -> Hint:
  """
  Return random letter from the word that has not been guessed yet.
  """
  # pre-conditions
  assert len(word) == WORD_LENGTH, "word not correct length"
  assert all(len(guess.word) == WORD_LENGTH for guess in guesses), "guesses not correct length"
  
  # creating set of guessed letters
  guessed_letters = set()
  for guess in guesses:
      for letter in guess.word:
          guessed_letters.add(letter)

  unguessed_letters = [letter for letter in word if letter not in guessed_letters]
  
  if not unguessed_letters:
      return None

  hint_letter = random.choice(unguessed_letters)

  # post-conditions
  assert hint_letter in word, "hint not in word"
  assert hint_letter not in guessed_letters, "hint already guessed"
  assert len(hint_letter) == 1, "hint not single letter"
  assert hint_letter.isalpha() or hint_letter.isupper(), "hint not a letter or not uppercase"

  return hint_letter

## Task 3 - Improving Check Guess/Check Letter

In [121]:
from collections import Counter

def check_guess(word: Word, guess: Word) -> List[Clue]:
  """
  Given the answer and a guess, compute the list of 
  clues corresponding to each letter.
  """
  # pre-conditions
  assert len(word) == WORD_LENGTH, "word not correct length"
  assert len(guess) == WORD_LENGTH, "guess not correct length"
  assert word in WORDS, "word not in word list"
  assert guess in WORDS, "guess not in word list"

  clues = [Clue.GREY] * WORD_LENGTH # set all clues to grey
  
  letter_counts = Counter(word) # returns dict with letter counts

  for i, letter in enumerate(guess): # check for green clues
      if letter == word[i]:
          clues[i] = Clue.GREEN
          letter_counts[letter] -= 1

  for i, letter in enumerate(guess): # check for yellow clues
      if letter != word[i] and letter in letter_counts and letter_counts[letter] > 0:
          clues[i] = Clue.YELLOW
          letter_counts[letter] -= 1

  # post-conditions
  assert len(clues) == WORD_LENGTH, "clues not correct length"
  assert all(clue in [Clue.GREEN, Clue.YELLOW, Clue.GREY] for clue in clues), "clues not valid"

  return clues

## Task 4 - Hard Guess

In [122]:
def hard_guess(self, word: Word):
  """
  Make a hard guess (All green letters from previous guesses must be in the correct place and yellow letters must be present).
  """

  word = word.upper()

  # pre-condition
  assert self.gstate == Gamestate.PLAYING, "game must be in play to make guess"
  assert len(word) == WORD_LENGTH, "word not correct length"
  assert word in WORDS, "word not in word list"

  green_letters = {}
  yellow_letters = set()
  for guess in self.guesses:
    for i, letter in enumerate(guess.word):
      if guess.clues[i] == Clue.GREEN:
        green_letters[i] = letter # add green letters to dict with index as key
      elif guess.clues[i] == Clue.YELLOW:
        yellow_letters.add(letter)

  # post-conditions
  assert all(word[i] == letter for i, letter in green_letters.items()), "Green letters not in word at correct position"
  assert all(letter in word for letter in yellow_letters), "Yellow letters not present in word"
  
  self.guesses.append(Guess(word, check_guess(self.answer, word)))
  if word == self.answer: self.gstate = Gamestate.WON
  elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

### Redefined Game Class
Adding get_hint and hard_guess functions to the game class.

In [123]:

class Game:
    answer: Word
    guesses: List[Guess]
    gstate: Gamestate

    def __init__(self, answer="STOUT"):
        """
        Constructor for game.
        """
        self.answer = answer
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def make_guess(self, word: Word):
        """
        Make a guess at the wordle.
        """
        # make guesses uppercase
        word = word.upper() 

        # pre-condition
        assert self.gstate == Gamestate.PLAYING and \
               len(self.guesses) < MAX_GUESSES and \
               word in WORDS, "pre-guess failed."
               
        self.guesses.append(Guess(word, check_guess(self.answer, word)))
        if word == self.answer: self.gstate = Gamestate.WON
        elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

    def print_state(self):
        """
        Prints a message to the user based on the current state of the game.
        """
        for guess in self.guesses: print(guess)
        if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
        elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
        else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

    def game_over(self) -> bool:
        """
        Yields true if the game is over (won or lost), false otherwise.
        """
        return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 

    def reset(self):
        """
        Reset the game by picking a new word, clearing the guess, and
        setting the state back to playing.
        """
        # pre-condition
        assert self.game_over(), "Cannot reset, game in play"
        self.answer = random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def hard_guess(self, word: Word):
        """
        Make a hard guess (All green letters from previous guesses must be 
        in the correct place and yellow letters must be present).
        """
        word = word.upper()

        # pre-condition
        assert self.gstate == Gamestate.PLAYING, "game must be in play to make guess"
        assert len(word) == WORD_LENGTH, "word not correct length"
        assert word in WORDS, "word not in word list"

        green_letters = {}
        yellow_letters = set()
        for guess in self.guesses:
            for i, letter in enumerate(guess.word):
                if guess.clues[i] == Clue.GREEN:
                    green_letters[i] = letter
                elif guess.clues[i] == Clue.YELLOW:
                    yellow_letters.add(letter)

        # post-conditions
        assert all(word[i] == letter for i, letter in green_letters.items()), "Green letters not in word at correct position"
        assert all(letter in word for letter in yellow_letters), "Yellow letters not present in word"
        
        self.guesses.append(Guess(word, check_guess(self.answer, word)))
        if word == self.answer: self.gstate = Gamestate.WON
        elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

    def get_hint(self) -> Hint:
        """
        Get a hint for the current game using auxiliary hint function.
        """
        # pre-condition
        assert self.gstate == Gamestate.PLAYING, "game must be in play to get hint"

        hint_letter = hint(self.answer, self.guesses)

        # post-condition
        assert hint_letter in self.answer, "hint not in answer"
        assert len(hint_letter) == 1, "hint not single letter"
        assert hint_letter.isalpha() or hint_letter.isupper(), "hint not a letter or not uppercase"

        return hint_letter


## Extension - Additional pre and post conditions

In [124]:
@dataclass
class Guess:
    """
    A class to represent a guess in Wordle, which is a Word 
    plus a clue for each letter.
    """
    word: Word
    clues: List[Clue]

    @property
    def word(self) -> Word:
        return self._word

    @word.setter
    def word(self, word: Word):
        # invariant
        assert len(word) == WORD_LENGTH, "Invariant violated: len(Guess.word) <> 5"
        assert word in WORDS, "Invariant violated: word not in word list"
        self._word = word

    @property
    def clues(self) -> List[Clue]:
        return self._clues

    @clues.setter
    def clues(self, clues: Clue):
        # invariant
        assert len(clues) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.clues) <> {WORD_LENGTH}"    
        assert all(clue in [Clue.GREEN, Clue.YELLOW, Clue.GREY] for clue in clues), "Invariant violated: clues not valid"          
        self._clues = clues

    def __repr__(self):
        """
        Custom representation for pretty printing.
        """
        cluestr = [str(self.word[i]) + ": " + \
            self.clues[i].name for i in range(WORD_LENGTH)]
        return f"{self.word}: {cluestr}"

In [125]:
import unittest

class TestCheckGuess(unittest.TestCase):
  """Test the check_guess function."""
  def test_correct(self):
    """Test a correct guess returns all green clues."""
    self.assertEqual(
      check_guess('STAND', 'STAND'), 
      [Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN], 
      "Expected [Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN] for check_guess('STAND', 'STAND')"
    )
  def test_partially_correct(self):
    """Test a partially correct guess returns the correct clues."""
    self.assertEqual(
      check_guess('STAND', 'STOUT'), 
      [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.GREY], 
      "Expected [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.GREY] for check_guess('STAND', 'STOUT')"
    )
    WORDS.append("STOUT")
    self.assertEqual(
      check_guess('TOAST', 'STOUT'), 
      [Clue.YELLOW, Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN], 
      "Expected [Clue.YELLOW, Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN] for check_guess('TOAST', 'STOUT')"
    ) 
    WORDS.remove("STOUT")

  def test_incorrect(self):
    """Test an incorrect guess returns all grey clues."""
    self.assertEqual(
      check_guess('WRONG', 'TESTS'), 
      [Clue.GREY, Clue.GREY, Clue.GREY, Clue.GREY, Clue.GREY],
      "Expected [Clue.GREY, Clue.GREY, Clue.GREY, Clue.GREY, Clue.GREY] for check_guess('WRONG', 'TESTS')"
    )

  def test_repeated_letters(self):
    """Test a guess with repeated letters returns the correct clues."""
    self.assertEqual(
      check_guess('BELLS', 'SELLS'), 
      [Clue.GREY, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN], 
      "Expected [Clue.GREY, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN] for check_guess('BELLS', 'SELLS')"
    )

  def test_guess_too_short(self):
    """Test a guess that is too short raises an AssertionError."""
    with self.assertRaises(AssertionError): 
      check_guess("WORD", "WORD")

  def test_guess_too_long(self):
    """Test a guess that is too long raises an AssertionError."""
    with self.assertRaises(AssertionError): 
      check_guess("TESTING", "TESTING")

  def test_words_not_in_list(self):
    """Test the word/guess not in the list raises an AssertionError."""
    with self.assertRaises(AssertionError): 
      check_guess("ASDFG", "TESTS")
    with self.assertRaises(AssertionError):
      check_guess("TESTS", "ASDFG")

  def test_special_characters(self):
    """Test a guess with special characters raises an AssertionError."""
    with self.assertRaises(AssertionError): 
      check_guess("TESTS", "TEST$")
    with self.assertRaises(AssertionError):
      check_guess("!@#$(", "&!@$#*")
    with self.assertRaises(AssertionError):
      check_guess("TESTS", "     ")

class TestGuess(unittest.TestCase):
  """Test the Guess class."""
  def test_guess(self):
    """Test the Guess class, which represents a guess in Wordle after it has been processed."""
    g = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
    self.assertEqual(g.word, "HELLO")

  def test_guess_clues(self):
    """Test the clues property of the Guess class."""
    g = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
    self.assertEqual(g.clues, [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])

  def test_invalid_word(self):
    """Test that an invalid word raises an AssertionError."""
    with self.assertRaises(AssertionError): 
      g = Guess("WORD", [Clue.GREEN] * WORD_LENGTH)
    
    with self.assertRaises(AssertionError):
      g = Guess("TESTING", [Clue.GREEN] * WORD_LENGTH)
    
    with self.assertRaises(AssertionError):
      g = Guess("ASDFG", [Clue.GREEN] * WORD_LENGTH)

  def test_invalid_clues(self):
    """Test that invalid clues raises an AssertionError."""
    with self.assertRaises(AssertionError): 
      g = Guess("TESTS", [Clue.GREEN])

class TestGame(unittest.TestCase):
  """Test the Game class."""
  def test_game_win(self):
    """Test the game is won after a correct guess."""
    game = Game("TESTS")
    game.make_guess("TESTS")
    self.assertEqual(game.gstate, Gamestate.WON, "Gamestate not WON after correct guess")

  def test_game_over(self):
    """Test the game is over after reaching the maximum number of guesses."""
    game = Game("TESTS")
    for i in range(MAX_GUESSES):
      game.make_guess("WRONG")
    self.assertEqual(game.gstate, Gamestate.LOST, "Gamestate not LOST after too many guesses")
    with self.assertRaises(AssertionError): 
      game.make_guess("WRONG")
  
  def test_reset(self):
    """Test the game is properly reset after a game over."""
    game = Game("TESTS")
    game.make_guess("TESTS")
    game.reset()
    self.assertEqual(game.gstate, Gamestate.PLAYING, "Gamestate not PLAYING after reset")
    self.assertEqual(game.guesses, [], "Guesses not empty after reset")

  def test_invalid_guess(self):
    """Test that an invalid guess raises an AssertionError."""
    game = Game("TESTS")
    with self.assertRaises(AssertionError): 
      game.make_guess("ASDFG")
    with self.assertRaises(AssertionError):
      game.make_guess("TESTING")
    with self.assertRaises(AssertionError):
      game.make_guess("WORD")
    with self.assertRaises(AssertionError):
      game.make_guess("     ")
    with self.assertRaises(AssertionError):
      game.make_guess("!%#*!")

  def test_max_guesses(self):
    """Test that the game handles guesses after max guesses are reached."""
    game = Game("TESTS")
    for i in range(MAX_GUESSES):
      game.make_guess("WRONG")
    with self.assertRaises(AssertionError): 
      game.make_guess("WRONG")

  def test_state(self):
    """Test game state correct after guess."""
    game = Game("TESTS")
    game.make_guess("WRONG")
    self.assertEqual(game.gstate, Gamestate.PLAYING, "Gamestate not PLAYING after guess")

    game.make_guess("TESTS")
    self.assertEqual(game.gstate, Gamestate.WON, "Gamestate not WON after correct guess")

class TestHint(unittest.TestCase):
  """Test the hint function."""
  def test_hint_normal(self):
    WORDS.append("ASDFG")
    WORDS.append("ASDFZ")
    game = Game("ASDFG")
    game.make_guess("ASDFZ")
    self.assertEqual(hint("ASDFG", game.guesses), "G", "Hint not G for ASDFG and ASDFZ")
    WORDS.remove("ASDFG")
    WORDS.remove("ASDFZ")
  
  def test_hint_no_letters(self):
    WORDS.append("ASDFG")
    WORDS.append("GFDAS")
    game = Game("ASDFG")
    game.make_guess("GFDAS")
    self.assertEqual(hint("ASDFG", game.guesses), None, "Hint not None for ASDFG and GFDAS")
    WORDS.remove("ASDFG")
    WORDS.remove("GFDAS")

  def test_hint_repeated_letters(self):
    game = Game("BELLS")
    game.make_guess("SELLS")
    self.assertEqual(hint("BELLS", game.guesses), "B", "Hint not B for BELLS and SELLS")

  def test_hint_type(self):
    game = Game("BELLS")
    game.make_guess("SELLS")
    self.assertEqual(len(hint("BELLS", game.guesses)), 1, "Hint is not one letter")
    self.assertTrue(hint("BELLS", game.guesses).isalpha(), "Hint is not a letter")
    self.assertTrue(hint("BELLS", game.guesses).isupper(), "Hint is not uppercase")

  def test_hint_multiple_possible(self):
    game = Game("HELLO")
    game.make_guess("TESTS")
    self.assertIn(hint("HELLO", game.guesses), ["H", "E", "L", "O"], "Hint not in word")

  def test_hint_not_in_previous_guesses(self):
    game = Game("HELLO")
    game.make_guess("HELPS")
    self.assertNotIn(hint("HELLO", game.guesses), ["H", "E", "L", "P", "S"], "Hint in previous guesses")

  def test_game_state(self):
    """Test the game state is correct after getting a hint."""
    game = Game("HELLO")
    game.make_guess("HELPS")
    game.get_hint()
    self.assertEqual(game.gstate, Gamestate.PLAYING, "Gamestate not PLAYING after getting hint")

class TestHardGuess(unittest.TestCase):
  """Test the hard_guess function."""
  def test_hard_guess_incorrect(self):
    """Test invalid hard guess"""
    game = Game("HELLO")
    game.make_guess("HELPS")
    with self.assertRaises(AssertionError): 
      game.hard_guess("TESTS")

  def test_hard_guess_correct(self):
    """Test valid hard guess"""
    game = Game("HELLO")
    game.make_guess("HELPS")
    game.hard_guess("HELLO")
    self.assertEqual(game.gstate, Gamestate.WON, "Gamestate not WON after correct hard guess")

unittest.main(argv=[''], exit=False)

...........................
----------------------------------------------------------------------
Ran 27 tests in 0.020s

OK
